<a href="https://colab.research.google.com/github/eren-ersoy/financial_forecasting/blob/main/financial_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install ta yfinance google-generativeai

import os
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from google import genai
import json
import ta
import os

# 1️⃣ Gemini API Key
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# 2️⃣ Hisse sembolü
symbol = input("Hisse senedi sembolünü gir (ör: AAPL, TSLA, GARAN.IS): ").upper()

# 3️⃣ Endeks belirleme
ticker = yf.Ticker(symbol)
try:
    exchange = ticker.info.get("exchange", "").upper()
except:
    exchange = ""

if symbol.endswith(".IS"):
    index_symbol = "XU100.IS"
elif exchange in ["NASDAQ", "NYSE", "AMEX"]:
    index_symbol = "^GSPC"
elif exchange in ["TSE", "JPX"]:
    index_symbol = "^N225"
elif exchange in ["EUREX", "XETRA"]:
    index_symbol = "^STOXX50E"
else:
    index_symbol = "^GSPC"

print(f"Hisse borsası: {exchange}, Endeks: {index_symbol}")

# 4️⃣ Tarih aralığı
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# 5️⃣ Veri çek
data = yf.download(
    symbol,
    start=start_date,
    end=end_date,
    auto_adjust=False,
    progress=False
)

index_data = yf.download(
    index_symbol,
    start=start_date,
    end=end_date,
    auto_adjust=False,
    progress=False
)

# 🔧 MultiIndex düzeltme
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

if isinstance(index_data.columns, pd.MultiIndex):
    index_data.columns = index_data.columns.get_level_values(0)

if data.empty:
    print("Veri bulunamadı.")
else:
    # Günlük getiri
    data["Daily Return"] = data["Close"].pct_change() * 100
    index_data["Index Daily Return"] = index_data["Close"].pct_change() * 100

    # === RSI ===
    close_prices = data["Close"]
    data["RSI"] = ta.momentum.RSIIndicator(close_prices, window=14).rsi()

    # === MACD ===
    macd = ta.trend.MACD(close_prices)
    data["MACD"] = macd.macd()
    data["MACD_Signal"] = macd.macd_signal()

    # === Momentum Yorumu ===
    latest_rsi = float(data["RSI"].iloc[-1])
    latest_macd = float(data["MACD"].iloc[-1])
    latest_signal = float(data["MACD_Signal"].iloc[-1])

    if latest_rsi > 50 and latest_macd > latest_signal:
        momentum_comment = "RSI ve MACD uyumlu: Boğa yönlü momentum"
    elif latest_rsi < 50 and latest_macd < latest_signal:
        momentum_comment = "RSI ve MACD uyumlu: Ayı yönlü momentum"
    elif latest_rsi > 50 and latest_macd < latest_signal:
        momentum_comment = "Negatif uyumsuzluk (RSI güçlü, MACD zayıf)"
    elif latest_rsi < 50 and latest_macd > latest_signal:
        momentum_comment = "Pozitif uyumsuzluk (RSI zayıf, MACD güçlü)"
    else:
        momentum_comment = "Momentum nötr"


    # === DÖNGÜ ANALİZİ (EMA50 / EMA200) ===
    data["EMA50"] = data["Close"].ewm(span=50).mean()
    data["EMA200"] = data["Close"].ewm(span=200).mean()

    data["Cycle"] = np.where(
        data["EMA50"] > data["EMA200"], "UP",
        np.where(data["EMA50"] < data["EMA200"], "DOWN", "FLAT")
    )

    cycles = []
    current_cycle = data["Cycle"].iloc[0]
    start_date_cycle = data.index[0]

    for i in range(1, len(data)):
        if data["Cycle"].iloc[i] != current_cycle:
            end_date_cycle = data.index[i - 1]
            duration = (end_date_cycle - start_date_cycle).days

            cycles.append({
                "Cycle": current_cycle,
                "Start": str(start_date_cycle.date()),
                "End": str(end_date_cycle.date()),
                "Duration_Days": duration
            })

            current_cycle = data["Cycle"].iloc[i]
            start_date_cycle = data.index[i]

    # son döngü
    cycles.append({
        "Cycle": current_cycle,
        "Start": str(start_date_cycle.date()),
        "End": str(data.index[-1].date()),
        "Duration_Days": (data.index[-1] - start_date_cycle).days
    })

    cycles_df = pd.DataFrame(cycles)

    cycle_summary = (
        cycles_df
        .groupby("Cycle")["Duration_Days"]
        .agg(Avg_Duration="mean", Median_Duration="median", Count="count")
        .round(1)
        .reset_index()
    )

    latest_cycle = cycles_df.iloc[-1]


    # ==== FIBONACCI DÜZELTMELERİ (365 GÜN) =========
    swing_low = float(data["Low"].min())
    swing_high = float(data["High"].max())
    price_range = swing_high - swing_low

    fib_levels = {
        "Swing_Low": round(swing_low, 2),
        "Fib_23.6": round(swing_high - price_range * 0.236, 2),
        "Fib_38.2": round(swing_high - price_range * 0.382, 2),
        "Fib_50.0": round(swing_high - price_range * 0.5, 2),
        "Fib_61.8": round(swing_high - price_range * 0.618, 2),
        "Fib_78.6": round(swing_high - price_range * 0.786, 2),
        "Swing_High": round(swing_high, 2)
    }

    # === Kanal Formasyonu (Linear Regression) ===
    import numpy as np

    window = 60  # son 60 gün
    recent = data.tail(window).copy()

    # Zaman ekseni
    x = np.arange(len(recent))
    y = recent["Close"].values

    # Lineer regresyon (y = mx + b)
    slope, intercept = np.polyfit(x, y, 1)

    # Trend yönü
    if slope > 0:
        channel_trend = "Yükselen Kanal (Uptrend)"
    elif slope < 0:
        channel_trend = "Alçalan Kanal (Downtrend)"
    else:
        channel_trend = "Yatay Kanal (Sideways)"

    # Kanal genişliği (standart sapma bazlı)
    residuals = y - (slope * x + intercept)
    channel_width = np.std(residuals)

    upper_channel = (slope * x[-1] + intercept) + channel_width
    lower_channel = (slope * x[-1] + intercept) - channel_width

    upper_channel = float(upper_channel)
    lower_channel = float(lower_channel)



    # === Üçgen Formasyonları (Triangle Detection) ===
    window = 60
    recent = data.tail(window).copy()

    x = np.arange(len(recent))

    # High ve Low için ayrı trendler
    highs = recent["High"].values
    lows = recent["Low"].values

    high_slope, high_intercept = np.polyfit(x, highs, 1)
    low_slope, low_intercept = np.polyfit(x, lows, 1)

    triangle_type = "Üçgen Yok"
    triangle_comment = "Belirgin bir sıkışma yapısı tespit edilmedi."

    # Eşik (çok küçük eğimleri yatay saymak için)
    slope_threshold = 1e-3

    if high_slope < -slope_threshold and low_slope > slope_threshold:
        triangle_type = "Simetrik Üçgen"
        triangle_comment = "Fiyat üstten düşen, alttan yükselen trend çizgileri arasında sıkışıyor."
    elif abs(high_slope) <= slope_threshold and low_slope > slope_threshold:
        triangle_type = "Yükselen Üçgen"
        triangle_comment = "Üst seviye yatay, dipler yükseliyor. Boğa yönlü kırılım potansiyeli."
    elif high_slope < -slope_threshold and abs(low_slope) <= slope_threshold:
        triangle_type = "Alçalan Üçgen"
        triangle_comment = "Alt seviye yatay, tepeler düşüyor. Ayı yönlü kırılım potansiyeli."


    # === Flama & Bayrak (Flag / Pennant Detection) ===
    flag_type = "Yok"
    flag_comment = "Trend devamına işaret eden net bir flama/bayrak yapısı yok."

    # Parametreler
    flagpole_window = 30
    consolidation_window = 20

    if len(data) > flagpole_window + consolidation_window:
        pole_data = data.iloc[-(flagpole_window + consolidation_window):-consolidation_window]
        cons_data = data.iloc[-consolidation_window:]

        pole_return = (pole_data["Close"].iloc[-1] / pole_data["Close"].iloc[0] - 1) * 100
        cons_range = (cons_data["High"].max() - cons_data["Low"].min()) / cons_data["Close"].mean()

        # Konsolidasyon trendi
        x_cons = np.arange(len(cons_data))
        cons_high_slope, _ = np.polyfit(x_cons, cons_data["High"], 1)
        cons_low_slope, _ = np.polyfit(x_cons, cons_data["Low"], 1)

        # Sert hareket + dar sıkışma şartı
        if abs(pole_return) > 8 and cons_range < 0.05:
            if cons_high_slope < 0 and cons_low_slope < 0:
                flag_type = "Ayı Bayrak"
                flag_comment = "Sert düşüş sonrası aşağı yönlü dar kanal. Trend devam potansiyeli."
            elif cons_high_slope > 0 and cons_low_slope > 0:
                flag_type = "Boğa Bayrak"
                flag_comment = "Sert yükseliş sonrası yukarı yönlü dar kanal. Trend devam potansiyeli."
            elif cons_high_slope < 0 and cons_low_slope > 0:
                flag_type = "Flama (Pennant)"
                flag_comment = "Sert hareket sonrası simetrik sıkışma. Kırılım trend yönünde beklenir."

   # === Takoz (Wedge Detection) ===
    wedge_type = "Yok"
    wedge_comment = "Net bir takoz formasyonu tespit edilmedi."

    wedge_window = 40

    if len(data) > wedge_window:
        wedge_data = data.iloc[-wedge_window:]

        x = np.arange(len(wedge_data))
        high_slope, _ = np.polyfit(x, wedge_data["High"], 1)
        low_slope, _ = np.polyfit(x, wedge_data["Low"], 1)

        slope_diff = abs(high_slope - low_slope)

        # Yükselen Takoz → Ayı
        if high_slope > 0 and low_slope > 0 and low_slope > high_slope and slope_diff > 0.0005:
            wedge_type = "Yükselen Takoz (Bearish)"
            wedge_comment = (
                "Fiyat yükselirken sıkışma artıyor. "
                "Yukarı yön zayıflıyor, aşağı kırılım riski var."
            )

        # Alçalan Takoz → Boğa
        elif high_slope < 0 and low_slope < 0 and high_slope > low_slope and slope_diff > 0.0005:
            wedge_type = "Alçalan Takoz (Bullish)"
            wedge_comment = (
                "Fiyat düşerken satış baskısı azalıyor. "
                "Yukarı yönlü dönüş potansiyeli var."
            )

    # === Çift Tepe / Çift Dip Detection ===
    double_pattern = "Yok"
    double_comment = "Çift tepe veya çift dip formasyonu tespit edilmedi."

    pattern_window = 60
    tolerance = 0.02  # %2 tolerans

    if len(data) > pattern_window:
        recent = data.iloc[-pattern_window:]

        # En yüksek iki tepe
        top1 = recent["High"].nlargest(2).values
        # En düşük iki dip
        bottom1 = recent["Low"].nsmallest(2).values

        # Çift Tepe
        if abs(top1[0] - top1[1]) / top1[0] < tolerance and latest_rsi < 50:
            double_pattern = "Çift Tepe (Bearish)"
            double_comment = (
                "Fiyat iki kez benzer zirve seviyesini test etti ancak yukarı yönlü devam edemedi. "
                "Momentum zayıflıyor, aşağı yönlü dönüş riski artıyor."
            )

        # Çift Dip
        elif abs(bottom1[0] - bottom1[1]) / bottom1[0] < tolerance and latest_rsi > 50:
            double_pattern = "Çift Dip (Bullish)"
            double_comment = (
                "Fiyat iki kez benzer dip seviyesinden destek buldu. "
                "Satış baskısı azalıyor, yukarı yönlü dönüş potansiyeli artıyor."
            )

    # === Destek & Direnç Hesabı (Son 60 Gün) ===
    window = 60

    recent_data = data.tail(window)

    support_level = float(recent_data["Low"].min())
    resistance_level = float(recent_data["High"].max())

    last_close = float(data["Close"].iloc[-1])

    # === Volatilite Bazlı Zaman Projeksiyonu ===
    data["Range"] = data["High"] - data["Low"]
    daily_volatility = data["Range"].rolling(14).mean().iloc[-1]

    # Zaman ölçekleri (iş günü bazlı)
    horizons = {
        "1 Ay": 21,
        "3 Ay": 63,
        "6 Ay": 126,
        "1 Yıl": 252
    }

    price_projections = {}

    for period, days in horizons.items():
        move = daily_volatility * (days ** 0.5)  # sqrt(time)
        price_projections[period] = {
            "min": round(last_close - move, 2),
            "max": round(last_close + move, 2)
        }



    # === AI için veri ===
    summary_data = data[[
        "Open","High","Low","Close",
        "Volume","Daily Return","RSI","MACD","MACD_Signal"
    ]].copy()

    summary_data.reset_index(inplace=True)
    summary_data["Date"] = summary_data["Date"].astype(str)

    summary_dict = summary_data.tail(120).to_dict(orient="records")  # 🔥 son 120 gün

    # === Prompt ===
    prompt = f"""
    ROLÜN:
    Sen deneyimli bir portföy yöneticisi ve teknik analistsin.
    Amacın kısa, net ve karar verilebilir bir fiyat öngörüsü sunmak.

    ⚠️ FORMAT ZORUNLULUĞU:
    Yanıtını AŞAĞIDAKİ SIRAYLA ve BAŞLIKLARLA ver.
    Bu sıranın dışına çıkma.
    Gereksiz uzun açıklama yapma.
    Önce özet, sonra detay.

    ========================
    {symbol}
    ========================

    📊 FİYAT FORECAST (ÖZET TABLO)
    (Yalnızca aşağıdaki volatilite bantlarını kullan)

    1 Ay  : {price_projections["1 Ay"]["min"]} – {price_projections["1 Ay"]["max"]}
    3 Ay  : {price_projections["3 Ay"]["min"]} – {price_projections["3 Ay"]["max"]}
    6 Ay  : {price_projections["6 Ay"]["min"]} – {price_projections["6 Ay"]["max"]}
    12 Ay : {price_projections["1 Yıl"]["min"]} – {price_projections["1 Yıl"]["max"]}

    📌 KISA TEKNİK YORUM (maksimum 7 madde)
    - Momentumun yönü (RSI + MACD)
    - Döngünün aşaması (erken / orta / geç)
    - En kritik destek
    - En kritik direnç
    - Ana risk
    - Ana fırsat
    - Genel teknik duruş (Boğa / Ayı / Nötr)

    📈 DETAYLI TEKNİK ANALİZ
    (Aşağıdaki verileri kullanarak açıkla)

    Teknik Momentum:
    - RSI (14): {latest_rsi:.2f}
    - MACD: {latest_macd:.4f}
    - Signal: {latest_signal:.4f}
    - Momentum Yorumu: {momentum_comment}

    Döngü (EMA50 / EMA200):
    - Mevcut Döngü: {latest_cycle["Cycle"]}
    - Döngü Süresi (gün): {latest_cycle["Duration_Days"]}
    - Tarihsel Döngü İstatistikleri:
    {json.dumps(cycle_summary.to_dict(orient="records"), indent=2)}

    Fibonacci Seviyeleri:
    {json.dumps(fib_levels, indent=2)}

    Kanal Yapısı:
    - Trend: {channel_trend}
    - Alt Bant: {lower_channel:.2f}
    - Üst Bant: {upper_channel:.2f}

    Formasyonlar:
    - Üçgen: {triangle_type} → {triangle_comment}
    - Flama/Bayrak: {flag_type} → {flag_comment}
    - Takoz: {wedge_type} → {wedge_comment}
    - Çift Tepe/Dip: {double_pattern} → {double_comment}

    Destek / Direnç:
    - Destek: {support_level:.2f}
    - Direnç: {resistance_level:.2f}

    ⚠️ KURALLAR:
    - Yeni seviye üretme
    - Tahmin ekleme
    - Sadece verilen sayısal verileri kullan
    - Abartılı dil kullanma
    - Finansal tavsiye verme
    """

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    print("\n🤖 AI Fiyat ve Momentum Analizi:\n")
    print(response.text)


Hisse senedi sembolünü gir (ör: AAPL, TSLA, GARAN.IS): TSLA
Hisse borsası: NMS, Endeks: ^GSPC

🤖 AI Fiyat ve Momentum Analizi:

📊 FİYAT FORECAST (ÖZET TABLO)

1 Ay  : 400.2 – 550.18
3 Ay  : 345.3 – 605.08
6 Ay  : 291.49 – 658.89
12 Ay : 215.4 – 734.98

📌 KISA TEKNİK YORUM (maksimum 7 madde)
- Momentumun yönü: Boğa
- Döngünün aşaması: Geç
- En kritik destek: 382.78
- En kritik direnç: 498.83
- Ana risk: Yükselen takoz formasyonunun aşağı yönlü kırılma potansiyeli
- Ana fırsat: Çift dip formasyonu sonrası yukarı yönlü hareket potansiyeli
- Genel teknik duruş: Nötr

📈 DETAYLI TEKNİK ANALİZ

Teknik Momentum:
RSI (14) değeri 57.27 seviyesinde olup, aşırı alım veya satım bölgesinde değildir. MACD (14.1895) sinyal hattının (12.4780) üzerinde seyretmektedir. Bu durum, RSI ve MACD verileriyle uyumlu olarak boğa yönlü bir momentumu işaret etmektedir.

Döngü (EMA50 / EMA200):
Mevcut döngü "UP" yönündedir ve 196 gündür devam etmektedir. Tarihsel döngü istatistiklerine göre, "UP" döngülerinin ortal